In [ ]:
from PIL import Image
import numpy as np
import pywt
import struct
import os


Read file and convert to greyscale

- 8 bit vs 16 bit?


In [ ]:

image_path = 'TestPhotos/GREYtest.png'
image = Image.open(image_path).convert('L')
image_array = np.array(image, dtype=np.uint8)

height, width = image_array.shape
image_size = os.path.getsize(image_path)
print(f"Loaded image: {width} x {height} pixels, grayscale,", image_size, "bytes")



Apply a multilevel DWT with haar wavelet and L levels, definitly not the most optimal wavelet, will use a better one down the line, also need to find the best value for L to use

The `coeffs` list structure:

coeffs[0] is the approximation coefficients at level L (cA_L)

coeffs[1:] are detail coefficients tuples for levels L, L-1, ..., 1:

e.g., coeffs[1] = (cH_L, cV_L, cD_L), coeffs[2] = (cH_{L-1}, cV_{L-1}, cD_{L-1}), ..., coeffs[L] = (cH_1, cV_1, cD_1)


In [ ]:
wavelet_name = 'haar'
L = 3 

coeffs = pywt.wavedec2(image, wavelet_name, level=L)

print(f"Decomposed into {len(coeffs)-1} levels of detail coefficients plus approximation.")
print("Approximation (cA_L) shape:", coeffs[0].shape)
for level, details in enumerate(coeffs[1:], start=1):
    cH, cV, cD = details
    print(f"Level {L+1-level} detail shapes (cH, cV, cD): {cH.shape}, {cV.shape}, {cD.shape}")


Use thresholding to zero out small coefficients (lossy compression) as they have little effect on final image

In [ ]:
threshold_fraction = 0.003

# find biggest coeff
all_coeffs = [coeffs[0]]  
for detail in coeffs[1:]:
    all_coeffs.extend(detail) 
max_coeff = max(np.max(np.abs(arr)) for arr in all_coeffs)
threshold_value = threshold_fraction * max_coeff

coeffs_thresh = [None] * len(coeffs)
cA = coeffs[0].copy()
cA[np.abs(cA) < threshold_value] = 0
coeffs_thresh[0] = cA

for i, detail in enumerate(coeffs[1:], start=1):
    cH, cV, cD = detail
    cH_th = cH.copy(); cH_th[np.abs(cH_th) < threshold_value] = 0
    cV_th = cV.copy(); cV_th[np.abs(cV_th) < threshold_value] = 0
    cD_th = cD.copy(); cD_th[np.abs(cD_th) < threshold_value] = 0
    coeffs_thresh[i] = (cH_th, cV_th, cD_th)

total_coeffs = sum(arr.size for arr in all_coeffs)
nonzero_coeffs = sum(np.count_nonzero(arr) for arr in [coeffs_thresh[0]] + [d for detail in coeffs_thresh[1:] for d in detail])
print(f"Total coefficients: {total_coeffs}, Non-zeros after threshold: {nonzero_coeffs}")


Quantise coefficients uniformly, from floating point numbers to intagers using 8-bit quantisation (256 levels)

In [ ]:
quant_levels = 256 

# min and max to scale
min_val = min(np.min(arr) for arr in all_coeffs)
max_val = max(np.max(arr) for arr in all_coeffs)
if max_val == min_val:
    step_size = 1.0
else:
    step_size = (max_val - min_val) / (quant_levels - 1)

# uniform quantise
quantized_coeffs = [None] * len(coeffs_thresh)

cA_th = coeffs_thresh[0]
qA = np.rint((cA_th - min_val) / step_size).astype(np.int16)
qA[cA_th == 0] = 0  # needa to check that i havent actaully set the threshold coefficients to zero twice lol
qA = np.clip(qA, 0, quant_levels-1)
quantized_coeffs[0] = qA


for i, detail in enumerate(coeffs_thresh[1:], start=1):
    cH_th, cV_th, cD_th = detail
    qH = np.rint((cH_th - min_val) / step_size).astype(np.int16)
    qV = np.rint((cV_th - min_val) / step_size).astype(np.int16)
    qD = np.rint((cD_th - min_val) / step_size).astype(np.int16)
    # second time being here haha ------------------------------------------
    qH[cH_th == 0] = 0
    qV[cV_th == 0] = 0
    qD[cD_th == 0] = 0
    qH = np.clip(qH, 0, quant_levels-1)
    qV = np.clip(qV, 0, quant_levels-1)
    qD = np.clip(qD, 0, quant_levels-1)
    quantized_coeffs[i] = (qH, qV, qD)

print("Quantization step size:", step_size)
print("Example quantized coeff range:", np.min(qA), "to", np.max(qA))

Serialise coefficients and metadata to store in a file, also store the following metadata

image dimensions
wavelet type
decomposition levels
thresholding and quantisation information (thresholding doesnt matter to decode)
length of encoded data


In [ ]:

flat_coeffs = []
# approximation
flat_coeffs.extend(quantized_coeffs[0].ravel().tolist())
# Append detail coefficients for each level
for detail in quantized_coeffs[1:]:
    qH, qV, qD = detail
    flat_coeffs.extend(qH.ravel().tolist())
    flat_coeffs.extend(qV.ravel().tolist())
    flat_coeffs.extend(qD.ravel().tolist())

total_values = len(flat_coeffs)
print("Total values to encode (should equal image size):", total_values)

# can we not entirely disregard the zeros coefficients instead of making them smaller in the entropy coding


apply entropy coding to losslessly reduce length of serialised string (from MA3K0 )

In [ ]:
def rle_encode(values, sentinel=0xFFFF):
    """Basic run-length encoding for a list of integers. 
    Uses `sentinel` followed by count to encode runs of zeros (length >=3)."""
    encoded = []
    i = 0
    n = len(values)
    while i < n:
        if values[i] != 0:
            # Non-zero value: encode it directly
            encoded.append(values[i])
            i += 1
        else:
            # Zero run detected
            j = i
            while j < n and values[j] == 0:
                j += 1
            run_len = j - i  # length of this zero-run
            if run_len < 3:
                # For short runs, output literal zeros (no compression gain for 1 or 2 zeros)
                encoded.extend([0] * run_len)
            else:
                # Use sentinel to encode a long run of zeros
                full_runs = run_len // 0xFFFF
                remainder = run_len % 0xFFFF
                # If run is very long, split into multiple sentinel segments
                for _ in range(full_runs):
                    encoded.append(sentinel)
                    encoded.append(0xFFFF)  # maximum count for a run in one segment
                if remainder > 0:
                    encoded.append(sentinel)
                    encoded.append(remainder)
            i = j
    return encoded

# RLE to the flat coefficient list
encoded_values = rle_encode(flat_coeffs)
print(f"RLE encoded length: {len(encoded_values)} (16-bit words)")


Now produce the .Job file, with the following data structure

width
height
dwt levels
wavelet name
threshold data
quantisation data
total coefficient count (should be width * height)
encoded data length
encoded coefficnent data


In [ ]:

def name_available(path: str, sep: str = "_") -> str:
    folder, filename = os.path.split(path)
    stem, ext = os.path.splitext(filename)

    candidate = path
    i = 1
    while os.path.exists(candidate):
        candidate = os.path.join(folder, f"{stem}{sep}{i}{ext}")
        i += 1
    return candidate

out_path = os.path.splitext(image_path)[0] + 'encoded.job'
output_filename = name_available(out_path)

with open(output_filename, "wb") as f:
    # width height and level
    f.write(struct.pack('<H', image_array.shape[1])) 
    f.write(struct.pack('<H', image_array.shape[0])) 
    f.write(struct.pack('<B', L))
    # Wavelet name
    wavelet_bytes = wavelet_name.encode('ascii')
    f.write(struct.pack('<B', len(wavelet_bytes)))
    f.write(wavelet_bytes)
    # Threshold fraction and quantization parameters
    f.write(struct.pack('<f', threshold_fraction))
    f.write(struct.pack('<f', float(min_val)))
    f.write(struct.pack('<f', float(step_size)))
    # Counts
    total_count = total_values
    encoded_count = len(encoded_values)
    f.write(struct.pack('<I', total_count))
    f.write(struct.pack('<I', encoded_count))
    # Encode list as 16-bit words
    data_array = np.array(encoded_values, dtype=np.uint16)
    f.write(data_array.tobytes())
